In [1]:
import numpy as np
from pc_kriging import PC_Kriging
import matplotlib.pyplot as plt
from scipy.stats import norm
from doepy import build
from scipy import optimize
import pandas as pd
from numpy import genfromtxt
import pickle

import matlab.engine
#to load matlab engine
eng = matlab.engine.start_matlab()

# adaptive learning - expected feasiability function --------------------------------------------------------------

def EFF(u,v,z):
    zl=-2*v
    zh=2*v
    return ((u-z)*( 2*norm.cdf((z-u)/v) - norm.cdf((zl-u)/v) - norm.cdf((zh-u)/v)) 
           -(v)*( 2*norm.pdf((z-u)/v) - norm.pdf((zl-u)/v) - norm.pdf((zh-u)/v))  
           +(2*v)*(norm.cdf((zh-u)/v) - norm.cdf((zl-u)/v)))

def U_function(u, v):
    return np.abs(u)/v

def LinearNorm(x,oldmin,oldmax,newmin,newmax):    # scaling linearly X to new domain limits
    return newmin + ((x-oldmin)*(newmax-newmin)/(oldmax-oldmin))


def VoronoiCell(x,xn):   #given x [single value] return the index of the closest xn [array]
    dist=PCK1.distance(x.reshape(1,-1),xn)
    return np.argmin(dist)

In [2]:
def simulator(xr1, xr2):
    #Recommended values... design domain
    # Amin1, Amax1 = [0.0,  45.0]   #Angles
    #Vmin2, Vmax2 = [0.5, 2.0]   #Velocity
    # dmin1, dmax1 = [1.3,  1.3]   #Leg diameter
    # dmin2, dmax2 = [0.4 , 0.8]   #Brace diameter
    # smin, smax = [287e6 , 347e6]   #317*10^6 flow stress
    # Emin, Emax = [210000e6, 210000e6] #210000*10^6 young modulus
    
    # fixing parameters for the solver
    Angle = 0.0
    Dleg = 1.3
    Dbra = 0.65
    YM = 210000e6

    Velocity = xr1.tolist()
    FlowStress = xr2.tolist()
    
    try:
        #wether xr is an array
        n = len(xr1)
        output = np.zeros((n, 3))
        
        for sample in range (0, len(output)):

            output[sample]  = np.array(eng.Main(Angle, Velocity[sample], Dleg, Dbra, FlowStress[sample], YM))

    except TypeError:
        #single point
        output = np.array(eng.Main(Angle, Velocity, Dleg, Dbra, FlowStress, YM))

    return output


In [3]:
config = {"pol_type": ['hermite', 'hermite']}   #design variables following normal distribution
PCK1 = PC_Kriging(config)
PCK_loo = PC_Kriging(config)    # for LOOCV with same 'config' as specified in the original model

## Initial input parameters

In [4]:
dim = 2       # dimensionality

x1mean, x1sigma = 3.0 , 0.6  # Velocity, normal distribution 
x2mean, x2sigma = 317e6 , 30e6  # FlowStress, normal distribution 

# objective function to optimize length scale --------------------------------------------------------
def L_Object (l):
    v = 5/2
    N = len(xn)
    R = PCK1.matern(xn , xn, l, v)
    detR = np.linalg.det(R)
    
    modelpar2 = PCK1.train(xn, yn, p, np.array([l,v]))    # returns B, sig2, InfoMatrix(phi) , PolyIndices(alpha)
    ### ------------------Theta_ by UQLab User Manual PCK(C. Lataniotis, D. Wicaksono, S. Marelli, B. Sudret)------------------------------
    sig2 = modelpar2[1].reshape(-1)
    # return 0.5*(np.log(detR)+ N*np.log(2*np.pi*sig2)+ N)

    ### ------------------Theta_ by MLE PCK(Schobi,Sudret,Wiart)------------------------------
    FB = PCK1.InfoMat @ modelpar2[0]
    ins = (yn-FB).reshape(-1)
    R_1 = np.linalg.inv(R)
    return ((ins.T) @ R_1 @ ins) * (1/N) * (detR**(1/N))

# Combined BDiffer training- U - LOO - 2 LS

In [18]:
number_experiments = 8    
number_active_points = 45

Critic_LS1 = 3
Critic_LS2 = 2

# kernel hyperparameters-------------------------------------
v = 5/2        #Gaussian process

#truncation term-------------------------------------
p_max = 4  #for each variable → same truncation , degree of expansion
    
results_file = '_BDiff_ULOO_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 5      # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    output = np.zeros((int(n), 3))        #observations [timeIndex, Penetration, MaxForce]

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    #simulator execution
    output = simulator(xr[:, 0], xr[:, 1]) #velocity and flowstress 
    
    #Output = [timeIndex, Penetration, MaxForce]
    yn = output[:, 1]

    #limit state definition
    yn_1 = Critic_LS1 - yn
    yn_2 = Critic_LS2 - yn

    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()

         ## Pool of samples for MCS -----------------------------------
        
        MCS_samples = 500000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma) 
        
        # #################################################################################################################
        # Active training of Limit State 1 ################################################################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
         
        yn = yn_1   #observations

        # OPTIMAL SURROGATE MODEL -----------------------------------

        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_1[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 
    
        # Pf estimation ----------------------------------------------       
        # surrogate 
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) ,'Pf_SuMO', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results)
        
        # #################################################################################################################
        # Active training of Limit State 2 ################################################################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2

        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) ,'Pf_SuMO', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results)
        
        # ##############################################################################
        
        if ( points == 0 ):   
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                 
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_1 == 0 ): 
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                 
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_2 == 0 ): 
            meanMC = meanMC_2
            varMC = varMC_2
            sumat = sumat_2
            opt = opt_2                 
            print ('LS 2')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        else:
            Pf1_new = fail_prob_SUMO_1
            Pf2_new = fail_prob_SUMO_2

            B1_old = - norm.ppf ( Pf1_old )
            B2_old = - norm.ppf ( Pf2_old )

            B1_new = - norm.ppf ( Pf1_new )
            B2_new = - norm.ppf ( Pf2_new )

            B_error1 = abs(B1_old - B1_new ) / B1_old
            B_error2 = abs(B2_old - B2_new ) / B2_old

            if (B_error1 > B_error2):

                meanMC = meanMC_1
                varMC = varMC_1
                sumat = sumat_1
                opt = opt_1                 
                print (B_error1, B_error2, 'LS 1')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2

            else:

                meanMC = meanMC_2
                varMC = varMC_2
                sumat = sumat_2
                opt = opt_2                 
                print (B_error1, B_error2, 'LS 2')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2
        
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+ sumat[voro, opt])

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), LOOCV.reshape(-1))
        
        candidate = MCinputs[np.argmin(U_f)]
        candidate_norm = MCinputs_norm[np.argmin(U_f)]
        
        xr = np.append(xr, candidate).reshape(-1,2)
        xn = np.append(xn, candidate_norm).reshape(-1,2)
        
        #simulator execution
        Single_yn = simulator(candidate[0], candidate[1]) #velocity and flowstress 
        
        #Output = [timeIndex, Penetration, MaxForce]
    
        yn_1 = np.append(yn_1, (Critic_LS1 - Single_yn[0,1]) )
        yn_2 = np.append(yn_2, (Critic_LS2 - Single_yn[0,1]) )

        print('number of training points: ', len(xn),'-------------------------------------------------')
        
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

Experiment:  1 #################################################################


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  del sys.path[0]


LS1:  Degree 1 e_LOO 1.757522614227588e-06 Pf_SuMO 0.000208 CoV 0.09800
LS2:  Degree 1 e_LOO 1.7575226142275581e-06 Pf_SuMO 0.131038 CoV 0.00360
LS 1
number of training points:  6 -------------------------------------------------
LS1:  Degree 1 e_LOO 4.565669633245639e-06 Pf_SuMO 0.000282 CoV 0.08420
LS2:  Degree 1 e_LOO 4.565669633245426e-06 Pf_SuMO 0.135366 CoV 0.00360
0.02304969037010084 0.017939810604656364 LS 1
number of training points:  7 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.447771302419484e-06 Pf_SuMO 0.000596 CoV 0.05790
LS2:  Degree 2 e_LOO 1.4477713016558675e-06 Pf_SuMO 0.142936 CoV 0.00350
0.06019478339754343 0.031013219675086568 LS 1
number of training points:  8 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.728247639025024e-06 Pf_SuMO 0.000542 CoV 0.06070
LS2:  Degree 2 e_LOO 1.728247639078647e-06 Pf_SuMO 0.142694 CoV 0.00350
0.00832462938255184 0.0010051288994853862 LS 1
number of training points:  9 --------

LS2:  Degree 2 e_LOO 2.6305827876254035e-10 Pf_SuMO 0.141614 CoV 0.00350
0.015589719797208499 0.001154346680069013 LS 1
number of training points:  36 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.7458822586885325e-09 Pf_SuMO 0.000564 CoV 0.05950
LS2:  Degree 2 e_LOO 1.7458252960978386e-09 Pf_SuMO 0.140888 CoV 0.00350
0.012049950680427633 0.003021319606131976 LS 1
number of training points:  37 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.996954479824266e-09 Pf_SuMO 0.000556 CoV 0.06000
LS2:  Degree 2 e_LOO 1.9969620536821543e-09 Pf_SuMO 0.142656 CoV 0.00350
0.0012447389218870478 0.007317315598331455 LS 2
number of training points:  38 -------------------------------------------------
LS1:  Degree 2 e_LOO 9.294820833876966e-09 Pf_SuMO 0.00057 CoV 0.05920
LS2:  Degree 2 e_LOO 9.294928325577282e-09 Pf_SuMO 0.141814 CoV 0.00350
0.0021649318270709175 0.0035027545267177266 LS 2
number of training points:  39 ----------------------------

LS2:  Degree 2 e_LOO 5.635808382202648e-08 Pf_SuMO 0.143004 CoV 0.00350
0.006058260590963007 0.00505017732265382 LS 1
number of training points:  21 -------------------------------------------------
LS1:  Degree 3 e_LOO 3.362012721747671e-08 Pf_SuMO 0.00054 CoV 0.06080
LS2:  Degree 3 e_LOO 3.362012719652701e-08 Pf_SuMO 0.141788 CoV 0.00350
0.008647459210495288 0.005062057611343811 LS 1
number of training points:  22 -------------------------------------------------
LS1:  Degree 3 e_LOO 8.897578402988157e-05 Pf_SuMO 0.000562 CoV 0.05960
LS2:  Degree 3 e_LOO 8.897578402989951e-05 Pf_SuMO 0.141426 CoV 0.00350
0.003461957201218175 0.001505016302441342 LS 1
number of training points:  23 -------------------------------------------------
LS1:  Degree 3 e_LOO 8.138497504005565e-05 Pf_SuMO 0.000566 CoV 0.05940
LS2:  Degree 3 e_LOO 8.13849750375217e-05 Pf_SuMO 0.1416 CoV 0.00350
0.0006180890500504905 0.0007226433720671429 LS 2
number of training points:  24 -------------------------------------

...

number of training points:  36 -------------------------------------------------
LS1:  Degree 2 e_LOO 2.379193911951251e-10 Pf_SuMO 0.000526 CoV 0.06160
LS2:  Degree 2 e_LOO 2.379196064068189e-10 Pf_SuMO 0.142582 CoV 0.00350
0.0009790622328332147 0.0010126254260867129 LS 2
number of training points:  37 -------------------------------------------------
LS1:  Degree 2 e_LOO 3.1489156544252887e-09 Pf_SuMO 0.000544 CoV 0.06060
LS2:  Degree 2 e_LOO 3.148918240253784e-09 Pf_SuMO 0.143116 CoV 0.00350
0.0029040746609469035 0.002214316977552335 LS 1
number of training points:  38 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.854365376162883e-09 Pf_SuMO 0.000602 CoV 0.05760
LS2:  Degree 2 e_LOO 1.854363094268687e-09 Pf_SuMO 0.14124 CoV 0.00350
0.008813843196861373 0.007821334453282333 LS 1
number of training points:  39 -------------------------------------------------
LS1:  Degree 2 e_LOO 3.6405194294206476e-09 Pf_SuMO 0.000514 CoV 0.06240
LS2:  Degree 2 e_LOO 3.6405

number of training points:  21 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.0742767692198631e-07 Pf_SuMO 0.000568 CoV 0.05930
LS2:  Degree 2 e_LOO 1.0742984849957307e-07 Pf_SuMO 0.14147 CoV 0.00350
0.002171386727276717 0.005041412032322945 LS 2
number of training points:  22 -------------------------------------------------
LS1:  Degree 2 e_LOO 3.4857517979160634e-07 Pf_SuMO 0.000524 CoV 0.06180
LS2:  Degree 2 e_LOO 3.485849207305437e-07 Pf_SuMO 0.141744 CoV 0.00350
0.007015847133467685 0.0011376518960173156 LS 1
number of training points:  23 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.6080891236628872e-07 Pf_SuMO 0.00051 CoV 0.06260
LS2:  Degree 2 e_LOO 1.6081380501999557e-07 Pf_SuMO 0.141466 CoV 0.00350
0.0023305574923428844 0.0011555856586379625 LS 1
number of training points:  24 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.696125545417163e-08 Pf_SuMO 0.000496 CoV 0.06350
LS2:  Degree 2 e_LOO 1.69

number of training points:  6 -------------------------------------------------
LS1:  Degree 1 e_LOO 9.601140741782403e-06 Pf_SuMO 0.000184 CoV 0.10420
LS2:  Degree 1 e_LOO 9.601140741782974e-06 Pf_SuMO 0.13825 CoV 0.00350
0.030053814719739398 0.007032607507244429 LS 1
number of training points:  7 -------------------------------------------------
LS1:  Degree 1 e_LOO 3.139238189214696e-08 Pf_SuMO 0.000368 CoV 0.07370
LS2:  Degree 1 e_LOO 3.1392381892169434e-08 Pf_SuMO 0.141656 CoV 0.00350
0.05228019590175478 0.014065762028807507 LS 1
number of training points:  8 -------------------------------------------------
LS1:  Degree 2 e_LOO 8.737021425664724e-06 Pf_SuMO 0.000556 CoV 0.06000
LS2:  Degree 2 e_LOO 8.73702142566359e-06 Pf_SuMO 0.14178 CoV 0.00350
0.03414430423091464 0.0005149742639879476 LS 1
number of training points:  9 -------------------------------------------------
LS1:  Degree 2 e_LOO 5.040568896763095e-06 Pf_SuMO 0.000562 CoV 0.05960
LS2:  Degree 2 e_LOO 5.040568896766941

LS1:  Degree 2 e_LOO 5.046410466673326e-09 Pf_SuMO 0.000566 CoV 0.05940
LS2:  Degree 2 e_LOO 5.046410375255251e-09 Pf_SuMO 0.141864 CoV 0.00350
0.007975318450987711 0.00020770387335969568 LS 1
number of training points:  37 -------------------------------------------------
LS1:  Degree 2 e_LOO 2.845705871210303e-09 Pf_SuMO 0.000614 CoV 0.05710
LS2:  Degree 2 e_LOO 2.8457066077636743e-09 Pf_SuMO 0.14231 CoV 0.00350
0.007122351582640604 0.0018505869254357241 LS 1
number of training points:  38 -------------------------------------------------
LS1:  Degree 1 e_LOO 9.84843746911002e-10 Pf_SuMO 0.000592 CoV 0.05810
LS2:  Degree 1 e_LOO 9.849071699867515e-10 Pf_SuMO 0.144146 CoV 0.00340
0.0032210168687923195 0.007591136432480649 LS 2
number of training points:  39 -------------------------------------------------
LS1:  Degree 2 e_LOO 3.7528152354507216e-08 Pf_SuMO 0.000554 CoV 0.06010
LS2:  Degree 2 e_LOO 3.752815240810765e-08 Pf_SuMO 0.141644 CoV 0.00350
0.005814815221873907 0.0104404005758

LS1:  Degree 3 e_LOO 4.6778462681424067e-07 Pf_SuMO 0.000568 CoV 0.05930
LS2:  Degree 3 e_LOO 4.677846268739698e-07 Pf_SuMO 0.141674 CoV 0.00350
0.0012228244278992527 0.0020752739627449414 LS 2
number of training points:  22 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.624583835250422e-07 Pf_SuMO 0.000526 CoV 0.06160
LS2:  Degree 2 e_LOO 1.624584586389435e-07 Pf_SuMO 0.141988 CoV 0.00350
0.0066853224235909284 0.0013034417496003172 LS 1
number of training points:  23 -------------------------------------------------
LS1:  Degree 1 e_LOO 1.9730741325629996e-07 Pf_SuMO 0.000588 CoV 0.05830
LS2:  Degree 1 e_LOO 1.9721490673335756e-07 Pf_SuMO 0.14228 CoV 0.00350
0.009645088127485423 0.0012119475111527162 LS 1
number of training points:  24 -------------------------------------------------
LS1:  Degree 2 e_LOO 2.1135145875699575e-07 Pf_SuMO 0.00055 CoV 0.06030
LS2:  Degree 2 e_LOO 2.1135204210325424e-07 Pf_SuMO 0.141928 CoV 0.00350
0.0058491342104823535 0.00146295

# Combined sequential training- U - LOO - 2 LS

In [ ]:
number_experiments = 10    
number_active_points = 45

Critic_LS1 = 3
Critic_LS2 = 2

# kernel hyperparameters-------------------------------------
v = 5/2        #Gaussian process
#truncation term-------------------------------------
p_max = 4  #for each variable → same truncation , degree of expansion
    
results_file = '_seq_ULOO_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 5      # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    output = np.zeros((int(n), 3))        #observations [timeIndex, Penetration, MaxForce]

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    #simulator execution
    output = simulator(xr[:, 0], xr[:, 1]) #velocity and flowstress 
    
    #Output = [timeIndex, Penetration, MaxForce]
    yn = output[:, 1]

    #limit state definition
    yn_1 = Critic_LS1 - yn
    yn_2 = Critic_LS2 - yn

    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()

         ## Pool of samples for MCS -----------------------------------
        
        MCS_samples = 500000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma) 
        
        # #################################################################################################################
        # Active training of Limit State 1 ################################################################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
         
        yn = yn_1   #observations

        # OPTIMAL SURROGATE MODEL -----------------------------------

        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_1[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 
    
        # Pf estimation ----------------------------------------------       
        # surrogate 
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) ,'Pf_SuMO', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results)
        
        # #################################################################################################################
        # Active training of Limit State 2 ################################################################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2

        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) ,'Pf_SuMO', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results)

        # ##############################################################################
        
        #Selecting the limit state to improve (1 each loop for sequential)
        
        if ( len(xn) % 2 == 0 ):   #even number
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                     
            print ('LS 1')

        else:    #odd number
            meanMC = meanMC_2
            varMC = varMC_2
            sumat = sumat_2
            opt = opt_2   
            print ('LS 2')
        
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+ sumat[voro, opt])

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), LOOCV.reshape(-1))
        
        candidate = MCinputs[np.argmin(U_f)]
        candidate_norm = MCinputs_norm[np.argmin(U_f)]
        
        xr = np.append(xr, candidate).reshape(-1,2)
        xn = np.append(xn, candidate_norm).reshape(-1,2)
        
        #simulator execution
        Single_yn = simulator(candidate[0], candidate[1]) #velocity and flowstress 
        
        #Output = [timeIndex, Penetration, MaxForce]
    
        yn_1 = np.append(yn_1, (Critic_LS1 - Single_yn[0,1]) )
        yn_2 = np.append(yn_2, (Critic_LS2 - Single_yn[0,1]) )

        print('number of training points: ', len(xn),'-------------------------------------------------')
        
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

# Active training- U - LOO - 2 LS

In [ ]:
number_experiments = 10    
number_active_points = 45

training_limitstate = 1

Critic_LS1 = 3
Critic_LS2 = 2

# kernel hyperparameters-------------------------------------
v = 5/2        #Gaussian process
#truncation term-------------------------------------
p_max = 4  #for each variable → same truncation , degree of expansion
    
results_file = '_AT'+ str(training_limitstate) +'_U_LOO_' + str(number_active_points)    # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 5      # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    output = np.zeros((int(n), 3))        #observations [timeIndex, Penetration, MaxForce]

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    #simulator execution
    output = simulator(xr[:, 0], xr[:, 1]) #velocity and flowstress 
    
    #Output = [timeIndex, Penetration, MaxForce]
    yn = output[:, 1]

    #limit state definition
    yn_1 = Critic_LS1 - yn
    yn_2 = Critic_LS2 - yn

    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()

         ## Pool of samples for MCS -----------------------------------
        
        MCS_samples = 500000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma) 
        
        # #################################################################################################################
        # Active training of Limit State 1 ################################################################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
         
        yn = yn_1   #observations

        # OPTIMAL SURROGATE MODEL -----------------------------------

        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_1[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 
    
        # Pf estimation ----------------------------------------------       
        # surrogate 
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) ,'Pf_SuMO', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results)
        
        # #################################################################################################################
        # Active training of Limit State 2 ################################################################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2

        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) ,'Pf_SuMO', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results)

        # ##############################################################################
        
        #Selecting the limit state to improve (1 each loop for sequential)
        
        #choosing the limit state to improve
        if (training_limitstate == 1):
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                     
            
        elif (training_limitstate == 2):
            meanMC = meanMC_2
            varMC = varMC_2
            sumat = sumat_2
            opt = opt_2
        
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+ sumat[voro, opt])

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), LOOCV.reshape(-1))
        
        candidate = MCinputs[np.argmin(U_f)]
        candidate_norm = MCinputs_norm[np.argmin(U_f)]
        
        xr = np.append(xr, candidate).reshape(-1,2)
        xn = np.append(xn, candidate_norm).reshape(-1,2)
        
        #simulator execution
        Single_yn = simulator(candidate[0], candidate[1]) #velocity and flowstress 
        
        #Output = [timeIndex, Penetration, MaxForce]
    
        yn_1 = np.append(yn_1, (Critic_LS1 - Single_yn[0,1]) )
        yn_2 = np.append(yn_2, (Critic_LS2 - Single_yn[0,1]) )

        print('number of training points: ', len(xn),'-------------------------------------------------')
        
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))